In [1]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score
import torch.nn as nn
from dataloader import load_data, CustomDataset
from model import init_model, train

In [20]:
# setup logger
import logging
from datetime import datetime
logfile = f"FeatureSelection_{datetime.now().strftime('%Y%m%d%H%M%S')}.log"
handler = logging.FileHandler(logfile)
handler.setFormatter(logging.Formatter("%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s"))
logger = logging.getLogger("logger")
logger.setLevel(logging.INFO)
logger.addHandler(handler)

In [2]:
# load all data
(
    train_data, val_data, test_data,
    patches, frames_idx, 
    county_data,
    counties_time_grid,
    precip_time_grid,
    tpi_array,
    landcover_array,
    scores_time_grid,
    night_time_grid,
    sentinel_time_grid
) = load_data()

Creating counties defor time grid: 28it [00:03,  7.04it/s]
Creating precipitations time grid: 28it [00:03,  8.26it/s]
Creating scores time grid: 28it [00:00, 34.45it/s]
Creating night lights time grid: 28it [00:06,  4.24it/s]
Creating sentinel time grid: 28it [00:03,  8.49it/s]
Computing patches: 100%|██████████| 32/32 [00:20<00:00,  1.58it/s]


In [3]:
def generate_dataloaders(channels):
    trainloader = torch.utils.data.DataLoader(
        CustomDataset(
            train_data, 
            patches, 
            frames_idx, 
            county_data,
            counties_time_grid,
            precip_time_grid,
            tpi_array,
            landcover_array,
            scores_time_grid,
            night_time_grid,
            sentinel_time_grid,
            channels
        ),
        batch_size=64,
        shuffle=True
    )

    valloader = torch.utils.data.DataLoader(
        CustomDataset(
            val_data, 
            patches, 
            frames_idx, 
            county_data,
            counties_time_grid,
            precip_time_grid,
            tpi_array,
            landcover_array,
            scores_time_grid,
            night_time_grid,
            sentinel_time_grid,
            channels
        ),
        batch_size=1000,
        shuffle=True
    )
    return trainloader, valloader

In [21]:
treshold_range = [29, 30]  # range(20, 40)

def evaluate_predictions(model, dataloader, treshold=None):
    # softmax function will be needed
    softmax_ = nn.Softmax(dim=1)
    # get prediction values as binary
    y_true = []
    y_pred = []
    for inputs, labels in dataloader:
        y_hat = model(inputs).detach()
        y_true.append(labels[:, 1, :, :].cpu())
        y_pred.append(softmax_(y_hat)[:, 1, :, :].cpu())
    # flatten
    y_true = np.concatenate(y_true).flatten() 
    y_pred = np.concatenate(y_pred).flatten()
    if treshold is None:
        # select probability treshold
        scores = []
        for p_treshold in tqdm(treshold_range, desc="Testing treshold values"):
            pt = p_treshold / 100
            scores.append((
                pt, 
                f1_score(
                    y_true, 
                    y_pred > pt
                )
            ))
        # sort by score and get from best treshold
        scores.sort(key=lambda i: i[1])
        best = scores[-1]
        print("Best treshold / score:", best)
        return best
    else:
        score = f1_score(
            y_true, 
            y_pred > treshold
        )
        print("Score:", score)
        return score

In [22]:
# number of epochs to train each model
n_epochs = 1

# list of all available channels
all_channels = list(range(6))

In [23]:

def run_selection_iteration(channels):
    # train model with defined starting channels
    print(f"\nEvaluating starting model:")
    trainloader, valloader = generate_dataloaders(channels)
    model, optimizer = init_model(len(channels))
    train(model, optimizer, n_epochs, trainloader, valloader)
    # basic_train_err = np.mean([i[0].cpu() for i in model.errs[-3:]])
    # basic_val_err = np.mean([i[1].cpu() for i in model.errs[-3:]])
    # basic_error = (basic_train_err, basic_val_err)
    # evaluate errors and estimate optimal treshold
    treshold, basic_train_score = evaluate_predictions(model, trainloader, treshold=None)
    basic_val_score = evaluate_predictions(model, valloader, treshold)
    basic_score = (basic_train_score, basic_val_score)
    logger.info(f"Basic scores: {basic_score}")

    # train model for other channels and save performance
    # c_errors = []
    c_scores = []
    for c in all_channels:
        if not(c in channels):
            print(f"\nStarting evaluation of additional feature ({c}):")
            logger.info(f"Starting evaluation of additional feature ({c})")
            c_trainloader, c_valloader = generate_dataloaders(channels+[c])
            c_model, c_optimizer = init_model(len(channels)+1)
            train(c_model, c_optimizer, n_epochs, c_trainloader, c_valloader)
            # avg_train_err = np.mean([i[0].cpu() for i in c_model.errs[-3:]])
            # avg_val_err = np.mean([i[1].cpu() for i in c_model.errs[-3:]])
            # c_errors.append((c, avg_train_err, avg_val_err))
            c_treshold, c_train_score = evaluate_predictions(c_model, c_trainloader, treshold=None)
            c_val_score = evaluate_predictions(c_model, c_valloader, c_treshold)
            c_scores.append((c, c_train_score, c_val_score))
            logger.info(f"Best treshold: {c_treshold}")
            logger.info(f"Train score: {c_train_score}")
            logger.info(f"Validation score: {c_val_score}")

    # get new channel that decreases (validation) loss the most
    # best_decrease = 0
    best_increase = 0
    best_channel = None
    # best_errors = None
    best_scores = None
    for (c, score_train, score_val) in c_scores:
        # decrease = float(basic_error[1] - err_val)
        increase = float(score_val - basic_val_score)
        if increase > best_increase:
            # best_decrease = decrease
            best_increase = increase
            best_channel = c
            # best_errors = (err_train, err_val)
            best_scores = (score_train, score_val)
        
    return best_channel, basic_score, best_scores

In [24]:
# start with basic features
channels = [0, 1, 2, 3]

In [25]:
# main selection loop
iter = 1
errors_hist = []
while True:
    print(f"\n\n Selection loop iteration {iter} | Channels: {channels}")
    logger.info(f"Selection loop iteration {iter} | Initial channels: {channels}")
    new_channel, starting_scores, final_scores = run_selection_iteration(channels)
    errors_hist.append((starting_scores, final_scores))
    if new_channel is None:
        logger.info("New iteration didn't find any channel that would increase model score")
        logger.info(f"Final channels: {channels}")
        break
    else:
        logger.info(f"New channel added: {new_channel} ({starting_scores} -> {final_scores})")
        channels.append(new_channel)
    iter += 1

print("\nFinal channels results:", channels)



 Selection loop iteration 1 | Channels: [0, 1, 2, 3]

Evaluating starting model:

Epoch 1


100%|██████████| 54/54 [00:12<00:00,  4.39it/s]


Epoch 1: Train Loss = 0.020667 | Validation Loss = 0.008864


Testing treshold values: 100%|██████████| 2/2 [00:08<00:00,  4.22s/it]


Best treshold / score: (0.3, 0.035106351003396416)
Score: 0.055720675546484986

Starting evaluation of additional feature (4):

Epoch 1


...

Best treshold / score: (0.29, 0.032940068691895114)
Score: 0.04990951192221995

Starting evaluation of additional feature (5):

Epoch 1


100%|██████████| 54/54 [00:12<00:00,  4.39it/s]


Epoch 1: Train Loss = 0.013099 | Validation Loss = 0.006234


Testing treshold values: 100%|██████████| 2/2 [00:08<00:00,  4.28s/it]


Best treshold / score: (0.29, 0.034085122098986224)
Score: 0.0526644082820204

Final channels results: [0, 1, 2, 3]


In [ ]:
# save results to file
errors_hist_df = []
for (e1, e2) in errors_hist:
    i = [float(e1[0]), float(e1[1])]
    if e2 is not None:
        i = i + [float(e2[0]), float(e2[1])]
    else:
        i = i + [np.nan, np.nan]
    errors_hist_df.append(i)
errors_hist_df = pd.DataFrame(
    errors_hist_df, 
    columns=["base_train", "base_val", "improved_train", "improved_val"]
)
errors_hist_df.to_csv("scores_history.csv", index=False)

In [ ]:
print("Final channels results:", channels)